# "Shazam"

Merci de remplir les informations ci-dessous, pour attribuer les notes :

Binome_C05

Rachid SAHLI, Ayoub Errahmani

## Consignes importantes

Vous aurez des manipulations à faire dans BigQuery, des requêtes à mettre au point, et des réponses à trouver. Je note les requêtes et les réponses. Pour les requêtes, vous avez 2 possibilités :
- soit les mettre au point directement dans le notebook
- soit les mettre au point dans l'interface de BigQuery, auquel cas **vous devrez les recopier dans le notebook**. BigQuery tout seul ne sauve pas les requêtes et je ne regarde pas ce qu'il y a dans BigQuery. On doit pouvoir réexécuter le notebook d'un coup sans erreur.

Il est conseillé de lire d'abord le notebook "Prise en main". Il y est fait régulièrement mention dans la suite.

La partie 1 est nécessaire pour faire les autres parties.

Pour faire la partie 3, vous aurez besoin de faire les parties 1 et 2 au moins jusqu'à la question "Enrichissement de la base de données". Exception : la toute dernière question de la partie 3 est totalement indépendante.

Si vous êtes vraiment bloqués : le dataset `shared` contient les tables que vous devez créer avec les requêtes, avec le préfixe `corrige__` (deux "tirets du 8"). Vous pouvez donc sauter une question en changeant le nom de la table (ex. `SELECT * FROM binome_xxx.raw_vectors` -> `SELECT * FROM shared.corrige__raw_vectors`). Le corrigé de l'UDF n'est pas donné en revanche.

#### Merci de ne faire qu'un notebook "TP3" par binôme.

#### A la fin du TP, avant de partir, sauvez ce notebook sur l'ordinateur, et envoyez-moi le fichier `TP3.ipynb` à l'adresse tvial@octo.com.

## Contexte et données

Vous développez un service de reconnaissance musicale (type Shazam), basé sur l’utilisation d’**embeddings** (vecteurs) censés représenter les “empreintes digitales” des morceaux. La recherche de morceaux proches d'un autre revient ainsi à calculer les distances entre des embeddings.

Les données sont constituées de morceaux composés par des artistes ; chaque morceau a préalablement été transformé en un certain nombre de vecteurs, dont le nombre peut varier. En effet un vecteur encode environ 10 s de musique, un morceau de 200 secondes aura ainsi une vingtaine de vecteurs, et un morceau de 400 secondes une quarantaine.

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/vectors.png)

### Informations sur les morceaux

Le premier jeu de données est un fichier `metadata.csv`, qui énumère les un peu plus de 1400 morceaux. Il contient une ligne par morceau avec les champs suivants séparés par des `“;”` :
- `song_id` : identifiant numérique du morceau
- `artist` : nom de l’artiste
- `title` : titre du morceau
- `duration` : durée du morceau, en secondes (NB : cette information est fictive)

### Vecteurs d'embedding

Le deuxième jeu de données est un ensemble de fichiers “JSONL”, c’est-à-dire des groupes de documents JSON stockés dans un même fichier avec un document par ligne. Ces fichiers correspondent aux vecteurs issus des morceaux. Il y a au total 1367 documents, regroupés en 18 fichiers `vectors_001.jsonl` à `vectors_018.jsonl`.

Il y a moins de documents que de morceaux (1367 contre 1418), car le calcul des embeddings a échoué pour quelques uns d'entre eux. Ce n'est pas gênant pour le TP.

Chaque document JSON (une ligne de fichier JSONL donc) est structuré ainsi :
- Un champ `“id”` qui contient l’ID du morceau (le même que dans le fichier `metadata.csv`)
- Un champ `“vectors”` qui est un tableau d’objets :
  - Chacun de ces objets a un seul champ `“vector”`, qui est un tableau de 16 nombres flottants
  - Tous les vecteurs sont donc de la même taille (16), mais il y a un nombre variable de tels vecteurs pour un morceau donné

En pratique, voici à quoi ressemble un fichier JSONL :
```
{“id”: 12, “vectors”: [{“vector”: [0.1, 0.2, ...]}, {“vector”: [0.4, 0.6, ...]}, ...]}
{“id”: 13, “vectors”: [{“vector”: [0.6, 0.5, ...]}, {“vector”: [0.7, 0.9, ...]}, ...]}
...
```

Les vecteurs ont été produits ainsi :
- lecture de fichiers MP3
- tranformation en embeddings avec le modèle [YAMNet](https://www.kaggle.com/models/google/yamnet)
- réduction du volume :
  - agrégation des fenêtres (10 vecteurs pour 1 seconde de musique => 1 vecteur pour 10 secondes, moyenne des 10 vecteurs)
  - ACP pour diminuer la taille des vecteurs de 1024 à 16

### Vecteurs de requêtes

Enfin, il y a 3 morceaux inconnus qui seront confrontés à la base de données des vecteurs : ce seront les "requêtes" de Shazam. Elles sont dans un seul fichier JSONL, avec la même structure que les précédents (le champ `"id"` représente l'ID de requête et non plus de morceau).

Toutes les données d’entrée sont dans le système de stockage objet de GCP, Google Cloud Storage, et nous allons les importer dans Big Query.

# Partie 1 : Manipulation dans BigQuery

Dans cette partie, on ne fait que des manipulations dans l'interface de BigQuery. Le notebook sert juste de guide et je n'attends pas de requête.

Prenez soin de respecter les étapes, car les données chargées serviront de base aux requêtes.

## Import des informations sur les morceaux

A l’aide de l’interface graphique, créer une table `metadata` dans votre dataset, dont les données se trouvent dans Google Cloud Storage.

- Après avoir choisi le type de source, un champ apparaît avec un bouton “PARCOURIR”. En cliquant dessus, il faut sélectionner le “bucket” (espace de stockage) `but-tp-shazam-datalake`, et à l’intérieur, le fichier `metadata.csv`.
- Le format doit être CSV
- Vous pouvez cocher “Détection automatique” dans la section “Schéma” pour que BigQuery trouve tout seul les champs et leurs types
- Attention, le séparateur de champs est un point-virgule dans le fichier, mais BigQuery suppose une virgule par défaut. Trouvez où changer ce paramètre
- Ne changez pas les autres paramètres par défaut
- Cliquez enfin sur “CREER LA TABLE” pour lancer le “job” de chargement

Si tout s’est bien passé, vous ne devez pas avoir d’erreurs. Sinon, vérifiez bien les informations.

En cliquant sur le nom de la table dans l’explorateur, vous pouvez voir la structure que Big Query a déduite, et en cliquant sur “PREVIEW”, avoir un extrait des données. Vérifiez bien que le contenu semble correct, sinon il faut supprimer la table (“Supprimer” depuis les 3 petits points de l’explorateur) et recommencer le chargement.

## Import des vecteurs d'embedding

Créer de même une nouvelle table, `raw_vectors`, qui va cette fois contenir les vecteurs bruts tirés des documents JSON.

- Ces données sont au format JSONL
- BigQuery est capable de détecter le schéma comme pour metadata
- Elles se trouvent dans le même bucket que `metadata.csv`, mais dans le répertoire `tracks`. Vous pouvez sélectionner un des fichiers du bucket, et remplacer ensuite son nom par un nom générique (ex. `vectors_001.jsonl` → `*.jsonl`) pour que BigQuery importe tout d’un coup

Comme précédemment, vérifiez le résultat via l’explorateur. La structure doit ressembler à ceci :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/struct_raw_vectors.png)

Et la prévisualisation n’est pas très lisible, à cause des tableaux imbriqués dans des objets :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/preview_raw_vectors.png)

## Import des requêtes

Procéder de même pour les requêtes, dans le répertoire `queries`, pour créer une table `raw_queries`.

# Partie 2 : Requêtes dans le notebook

Dans cette partie, il faut écrire les requêtes dans le notebook.

## Configuration de l'extension BigQuery.

In [3]:
%load_ext bigquery_magics

The bigquery_magics extension is already loaded. To reload it, use:
  %reload_ext bigquery_magics


In [8]:
import bigquery_magics
bigquery_magics.context.project = 'but-tp'

## Transposition des vecteurs

Actuellement, les données de vecteurs dans la table ont un modèle calqué sur celui des fichiers, qui n’est donc pas très pratique. On cherche à les “dénormaliser”, c’est-à-dire à transformer chaque ligne (qui correspond à un morceau) en autant de lignes qu’il y a de vecteurs pour son morceau, en répétant l’ID du morceau.

Schématiquement, cela revient à faire une opération de ce style pour chaque morceau (ici celui d’ID 42) :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/transposition.png)

Les ID sont répétés autant de fois qu’il y a de fenêtres et donc de vecteurs. Le nouveau champ `window_id` est le numéro de la fenêtre dans le morceau, i.e. la position du vecteur dans le tableau (le premier ayant la position 0).

Mettre au point une requête `SELECT` qui fait cette transposition, puis créer une table `flat_vectors` avec le résultat. La table doit avoir la structure suivante :
- `id` : l’ID du morceau
- `window_id` : numéro de la fenêtre
- `vector` : un vecteur unique = un tableau à 16 flottants

Vous pouvez vous aider du notebook de "Prise en main", dans la partie III. Il faut en plus accéder au sous-tableau `vector` qui se situe à l’intérieur des éléments du tableau principal (analogue à `myarray` dans l’exemple). L’équivalent de `element` est alors un objet JSON, dont on peut extraire l’attribut vector grâce à la syntaxe suivante : `element.vector` (à utiliser dans la liste des colonnes après le `SELECT`).

In [31]:
%%bigquery
CREATE TABLE binome_c05.flat_vectors AS
SELECT id, window_id, vector.vector AS vector
FROM binome_c05.raw_vectors
CROSS JOIN UNNEST(vectors) AS vector
WITH OFFSET AS window_id;

Query is running:   0%|          |

Downloading:   0%|          |

,id,window_id,vector
0,81,0,"[-7.690122127532959, 2.078136920928955, -1.010..."
1,81,1,"[0.07823469489812851, -0.36372220516204834, -0..."
2,81,2,"[0.9122859835624695, -0.10936319828033447, 0.1..."
3,81,3,"[0.7467789053916931, -0.6708750128746033, -0.3..."
4,81,4,"[1.1462376117706299, 0.42200031876564026, -0.1..."
...,...,...,...
70119,80,45,"[-1.41603684425354, 0.8403050899505615, -0.248..."
70120,80,46,"[-0.7067599892616272, -0.8691008687019348, -1...."
70121,80,47,"[-1.5009812116622925, -0.9736199975013733, -0...."
70122,80,48,"[-1.7764664888381958, -1.6022533178329468, -1...."


Vous devez obtenir 70 124 lignes, et l’aperçu dans BigQuery doit ressembler à ceci (l’ordre des morceaux et fenêtres est arbitraire) :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/preview_flat_vectors.png)

## Transposition des vecteurs de `raw_queries`

Appliquer la même méthode pour créer une table `flat_queries`, à partir de `raw_queries`. La structure doit être la même.

In [32]:
%%bigquery
CREATE TABLE binome_c05.flat_queries AS
SELECT id, window_id, vector.vector AS vector
FROM binome_c05.raw_queries
CROSS JOIN UNNEST(vectors) AS vector
WITH OFFSET AS window_id;

Query is running:   0%|          |

""


La table doit contenir 36 éléments.

## Enrichissement de la base de données

Maintenant que nous avons des vecteurs à plat, nous allons enrichir la table avec les métadonnées des morceaux.

Créer une table `vectors_with_metadata`, avec comme structure :
- `song_id` (provient des deux tables)
- `window_id` (provient de `flat_vectors`)
- `artist` (provient de `metadata`)
- `title` (provient de `metadata`)
- `duration` (provient de `metadata`)
- `vector` (provient de `flat_vectors`)

In [54]:
%%bigquery
CREATE TABLE binome_c05.vectors_with_metadata AS
SELECT flat_v.id AS song_id, 
    flat_v.window_id, meta.artist, meta.title, meta.duration, flat_v.vector  
FROM binome_c05.flat_vectors AS flat_v
JOIN binome_c05.metadata AS meta
ON flat_v.id = meta.song_id; 

Query is running:   0%|          |

""


**Question** : à combien de lignes peut-on s'attendre comme résultat, et pourquoi ?

On prend toutes les observations de la table flat_vectors qui fait 70 124 lignes et on leur associe les métadonnées depuis metadata via song_id. Comme metadata a une seule ligne par song_id, chaque ligne de flat_vectors trouve au plus une correspondance dans metadata.
Le nombre de lignes dans le résultat sera donc égal au nombre de lignes dans flat_vectors soit 70 124.

## Création d'une UDF

Pour les besoins de notre clone de Shazam, il faut être en mesure de calculer la distance euclidienne entre deux vecteurs de taille identique. Nous allons pour cela créer une UDF : _User-Defined Function_.

Dans le notebook de prise en main, vous avez un exemple d’UDF qui met en rapport les éléments de 2 vecteurs passés en paramètre, comme une sorte de jointure. Elle est aussi disponible dans vos environnements sous le nom `shared.join_vectors`.

Pour cette question, il est demandé de créer une autre UDF, `euclidean2`, qui retourne un flottant et non une table virtuelle, en l’occurrence le carré de la distance euclidienne entre 2 vecteurs qui lui sont passés en paramètre. Vous pourrez utiliser directement la fonction `join_vectors`, comme si son résultat était une table SQL avec 2 colonnes, une pour chaque vecteur, et une ligne par paire d’éléments. Ou vous pourrez vous en inspirer pour écrire `euclidean2` de zéro, comme vous le souhaitez.

Pour rappel, si $(x_i)$ et $(y_i)$ sont deux vecteurs, le carré de cette distance est $\sum_{i}{(x_i-y_i)^{2}}$. On ne cherche pas à appliquer une racine carrée pour avoir la distance absolue, le carré suffit.

%%bigquery
CREATE OR REPLACE FUNCTION binome_xxx.euclidean2 ...

In [55]:
%%bigquery
CREATE FUNCTION binome_c05.euclidean2(vec1 ARRAY<FLOAT64>, vec2 ARRAY<FLOAT64>) 
RETURNS FLOAT64 
AS (
  (SELECT SUM(POW(v1 - v2, 2))
   FROM UNNEST(vec1) AS v1 WITH OFFSET pos
   JOIN UNNEST(vec2) AS v2 WITH OFFSET pos USING (pos))
);

Query is running:   0%|          |

""


Test de la fonction :

In [56]:
%%bigquery
SELECT binome_c05.euclidean2([3., 1., 2.], [4., 5., 6.])

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,33.0


Le résultat doit être 33.

## Calcul des distances par fenêtre

Ecrire une requête qui calcule, pour chaque morceau de `vectors_with_metadata`, le carré de la distance entre tous ses vecteurs et tous ceux de la table `flat_queries` (produit cartésien). Le résultat doit avoir les colonnes suivantes :
- `song_id_ref` (ID du morceau de référence provenant de `vectors_with_metadata`)
- `window_id_ref` (ID de la fenêtre du vecteur du morceau de référence)
- `query_id` (ID de la requête provenant de `flat_queries`)
- `window_id_query` (ID de la fenêtre du vecteur issu de `flat_queries`)
- `artist_ref`, `title_ref`, `duration_ref` : informations du morceau de référence
- `distance2` (carré de la distance)

Dans ce qui précède, `_ref` désigne donc chaque morceau de la base de référence (attention de ne pas mélanger).

Il n’est pas demandé de créer une table, en revanche il faut bien copier la requête dans la cellule ci-dessous.

Gardez la clause `LIMIT 100`, elle permet de n'envoyer que quelques lignes (100) au notebook ; on n'est pas intéressé par le détail pour l'instant.

In [62]:
%%bigquery
WITH filtered_vwm AS (
    SELECT *
    FROM binome_c05.vectors_with_metadata
    WHERE song_id = 43
),
filtered_fq AS (
    SELECT *
    FROM binome_c05.flat_queries
    WHERE id = 2
)
SELECT 
    vwm.song_id AS song_id_ref, 
    vwm.window_id AS window_id_ref,    
    fq.id AS query_id, 
    fq.window_id AS window_id_query, 
    vwm.artist AS artist_ref,          
    vwm.title AS title_ref, 
    vwm.duration AS duration_ref,      
    binome_c05.euclidean2(vwm.vector, fq.vector) AS distance2
FROM 
    filtered_vwm AS vwm
CROSS JOIN 
    filtered_fq AS fq
ORDER BY 
    vwm.window_id,    
    fq.window_id 
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,song_id_ref,window_id_ref,query_id,window_id_query,artist_ref,title_ref,duration_ref,distance2
0,43,0,2,0,Indochine,Paradize,450,735.926447
1,43,0,2,1,Indochine,Paradize,450,113.464269
2,43,0,2,2,Indochine,Paradize,450,105.423871
3,43,0,2,3,Indochine,Paradize,450,109.091173
4,43,0,2,4,Indochine,Paradize,450,95.761539
...,...,...,...,...,...,...,...,...
95,43,0,2,95,Indochine,Paradize,450,83.696582
96,43,0,2,96,Indochine,Paradize,450,60.185806
97,43,0,2,97,Indochine,Paradize,450,63.541962
98,43,0,2,98,Indochine,Paradize,450,63.741539


Pour contrôler, voici un les premières lignes du résultat pour `song_id_ref = 43` et  `query_id = 2`, triées par `window_id_ref` et `window_id_query` :

![](https://raw.githubusercontent.com/tvial/BUT-R6.01-TP3/refs/heads/main/images/check_query1.png)

**Question** : comment peut-on prédire le nombre de lignes du résultat ?

On sait que nombre de lignes dans le résultat de la jointure est égal au produit du nombre de lignes de chaque table après filtrage.
Car CROSS JOIN produit un produit cartésien où chaque ligne de la table 1 est combinée avec chaque ligne de la table 2.
Le nombre final de lignes après jointure est donc le produit du nombre de lignes après filtrage.

## Rapprochement des fenêtres

Réutiliser la requête de la question précédente **sans la clause `LIMIT`** dans une clause `WITH` (voir notebook "Prise en main"), pour produire une requête qui calcule, pour chaque fenêtre de `queries` et chaque morceau de référence, la distance à la fenêtre la plus proche du morceau de référence.

Structure attendue du résultat :
- `query_id` (ID de la requête)
- `song_id_ref` (ID du morceau de référence comparé)
- `artist_ref`, `title_ref`, `duration_ref` : informations du morceau de référence
- `window_id_query` (ID de la fenêtre du vecteur requêté)
- `min_distance2` (carré de la distance la plus faible parmi les fenêtres du morceau de référence, pour un ensemble `query_id` + `window_id_query` + morceau de référence donné)

Rappel : il faut retirer la clause `LIMIT` de la requête réutilisée, mais on en ajoute une sur le résultat global.

In [63]:
%%bigquery
WITH filtered_vwm AS (
    SELECT *
    FROM binome_c05.vectors_with_metadata
    WHERE song_id = 43
),
filtered_fq AS (
    SELECT *
    FROM binome_c05.flat_queries
    WHERE id = 2
),
distance_calculation AS (
    SELECT 
        vwm.song_id AS song_id_ref, 
        fq.id AS query_id, 
        fq.window_id AS window_id_query, 
        vwm.window_id AS window_id_ref, 
        binome_c05.euclidean2(vwm.vector, fq.vector) AS distance2,
        vwm.artist AS artist_ref,          
        vwm.title AS title_ref, 
        vwm.duration AS duration_ref
    FROM 
        filtered_vwm AS vwm
    CROSS JOIN 
        filtered_fq AS fq
)
SELECT 
    query_id, 
    song_id_ref, 
    artist_ref, 
    title_ref, 
    duration_ref, 
    window_id_query, 
    MIN(distance2) AS min_distance2
FROM 
    distance_calculation
GROUP BY 
    query_id, 
    song_id_ref, 
    artist_ref, 
    title_ref, 
    duration_ref, 
    window_id_query
ORDER BY 
    query_id, 
    song_id_ref, 
    window_id_query
LIMIT 100;

Query is running:   0%|          |

Downloading:   0%|          |

,query_id,song_id_ref,artist_ref,title_ref,duration_ref,window_id_query,min_distance2
0,2,43,Indochine,Paradize,450,0,352.799104
1,2,43,Indochine,Paradize,450,1,12.102091
2,2,43,Indochine,Paradize,450,2,10.975885
3,2,43,Indochine,Paradize,450,3,10.824390
4,2,43,Indochine,Paradize,450,4,9.035571
...,...,...,...,...,...,...,...
95,2,43,Indochine,Paradize,450,95,4.143783
96,2,43,Indochine,Paradize,450,96,5.881379
97,2,43,Indochine,Paradize,450,97,6.017792
98,2,43,Indochine,Paradize,450,98,8.472215


## Détermination des morceaux les plus proches

Une fois ceci mis au point, proposer les morceaux les plus probables pour les 3 requêtes de `flat_queries`.

Pour ce faire, on peut :
- encapsuler la requête précédente dans une nouvelle clause `WITH`
- ... et l’utiliser dans une agrégation en calculant, pour chaque requête et chaque morceau de référence, la somme des `min_distance2`. Un tri sur le couple (somme des `min_distance2`, `query_id`) devrait vous remonter les morceaux candidats en premier.

Ainsi, la "distance" entre 2 morceaux est la somme des distances minimales entre les fenêtres des morceaux.

**Vérifiez que votre requête ne contient plus aucune clause `LIMIT`**.

In [93]:
%%bigquery
WITH filtered_vwm AS (
    SELECT *
    FROM binome_c05.vectors_with_metadata
),
filtered_fq AS (
    SELECT *
    FROM binome_c05.flat_queries
    WHERE id IN (1, 2, 3)
),
distance_calculation AS (
    SELECT 
        vwm.song_id AS song_id_ref, 
        fq.id AS query_id, 
        fq.window_id AS window_id_query, 
        vwm.window_id AS window_id_ref, 
        binome_c05.euclidean2(vwm.vector, fq.vector) AS distance2,
        vwm.artist AS artist_ref,          
        vwm.title AS title_ref, 
        vwm.duration AS duration_ref
    FROM 
        filtered_vwm AS vwm
    CROSS JOIN 
        filtered_fq AS fq
),
min_distance_calculation AS (
    SELECT 
        query_id, 
        song_id_ref, 
        artist_ref, 
        title_ref, 
        duration_ref, 
        window_id_query, 
        MIN(distance2) AS min_distance2
    FROM 
        distance_calculation
    GROUP BY 
        query_id, 
        song_id_ref, 
        artist_ref, 
        title_ref, 
        duration_ref, 
        window_id_query
),
total_distance_calculation AS (
    SELECT 
        query_id, 
        song_id_ref, 
        artist_ref, 
        title_ref, 
        duration_ref, 
        SUM(min_distance2) AS total_distance
    FROM 
        min_distance_calculation
    GROUP BY 
        query_id, 
        song_id_ref, 
        artist_ref, 
        title_ref, 
        duration_ref
)
SELECT 
    song_id_ref, 
    artist_ref, 
    title_ref, 
    query_id, 
    total_distance
FROM 
    total_distance_calculation
ORDER BY 
    total_distance ASC,
    query_id,
    song_id_ref
LIMIT 3;

Query is running:   0%|          |

Downloading:   0%|          |

,song_id_ref,artist_ref,title_ref,query_id,total_distance
0,237,Serge Gainsbourg,Sous Le Soleil Exactement,3,0.108743
1,56,Queen,Seven Seas Of Rhye,1,2.137338
2,522,W. A. Mozart,"Concerto for Piano in D minor, K466 - Allegro",2,2.282579


# Réponse :
### morceau de la requête n°1 : Sous Le Soleil Exactement
### morceau de la requête n°2 : Seven Seas Of Rhye
### morceau de la requête n°3 : Concerto for Piano in D minor, K466 - Allegro

# Partie 3 - Machine learning

Nous continuons l’exploration des données avec la création d’un modèle de machine learning, dans BigQuery. Ce sera un modèle de clustering, opérant sur les vecteurs.

On aurait aimé réutiliser notre fonction de distance `euclidean2`, mais BigQuery ML n’offre malheureusement pas la possibilité de personnaliser celle-ci. On va donc utiliser la distance euclidienne fournie.

## Entraînement du modèle

Le K-means de BigQuery suppose que les vecteurs sont sous forme tabulaire, et pas de tableaux imbriqués comme on en a manipulé jusqu’à présent. il faut transformer les données avant d’entraîner le modèle (on peut voir ça comme une étape de feature engineering).

Nous devons donc “pivoter” les vecteurs pour en faire des colonnes. Mais il faut d’abord les “pivoter” en ligne ! On fournit la requête de pivotage dans le notebook de prise en main, vous pouvez l’exécuter pour voir son résultat.

Maintenant que l’on sait pivoter les données, entraîner un modèle de type K-means, en utilisant le squelette proposé par le notebook de prise en main, sur tout le jeu de données pivoté. **La transformation doit spécifier que seules les colonnes V0 à V15 sont utilisées**.

Les options du modèle doivent être :
- `MODEL_TYPE = 'KMEANS'`
- `NUM_CLUSTERS = 5`
- `KMEANS_INIT_METHOD = 'KMEANS++'`
- `DISTANCE_TYPE = 'euclidean'`
- `STANDARDIZE_FEATURES = FALSE`

Vous pouvez aller dans l'interface de BigQuery voir le détail du modèle et accéder à des statistiques sur l’entraînement, les centroïdes des clusters, ...

In [126]:
%%bigquery
WITH unfolded AS (
  SELECT
      vmd.* EXCEPT (vector),
      element,
      'V' || CAST(position AS STRING) AS position
  FROM binome_c05.vectors_with_metadata AS vmd
  CROSS JOIN UNNEST(vmd.vector) AS element WITH OFFSET position
)
SELECT *
FROM unfolded
PIVOT (
  MIN(element)
  FOR position IN (
    'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
  )
);

Query is running:   0%|          |

Downloading:   0%|          |

,song_id,window_id,artist,title,duration,V0,V1,V2,V3,V4,...,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,157,2,AC-DC,Goodbye & good riddance to bad luck,456,1.824852,-0.289892,-0.388033,1.584664,-0.456470,...,-0.523962,-0.637617,-0.055382,0.296999,0.083297,0.138613,0.132536,0.056009,0.350568,-0.204127
1,157,13,AC-DC,Goodbye & good riddance to bad luck,456,-0.150979,-1.163190,-0.256033,0.952783,-0.148143,...,-0.007819,-0.386219,0.471329,-0.406921,0.123514,0.134665,-0.124617,-0.188424,-0.097516,0.080401
2,157,23,AC-DC,Goodbye & good riddance to bad luck,456,0.130720,0.306384,-0.412713,0.654630,-0.227542,...,-0.672093,0.651712,-0.758895,0.141295,-0.686246,-0.128747,-0.176800,-0.476764,-0.349506,-0.053393
3,157,7,AC-DC,Goodbye & good riddance to bad luck,456,-1.656955,1.513906,-0.887978,-0.292419,0.178305,...,-0.408847,-0.400397,-0.183566,0.084843,-0.198751,-0.034689,-0.319834,0.258719,0.155988,0.334232
4,157,25,AC-DC,Goodbye & good riddance to bad luck,456,3.363889,0.052055,0.209891,0.008791,0.153389,...,0.243926,0.646251,-0.503002,0.002912,-0.278426,-0.017344,-0.395501,-0.090363,-0.226088,-0.039514
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70119,1358,20,ZZTop,Lovething,205,-0.130061,-0.667420,-0.285901,-0.300582,1.111448,...,0.624330,-0.485531,0.226437,-0.147430,0.306496,-0.150449,-0.800183,-0.445903,0.097990,0.148802
70120,1358,27,ZZTop,Lovething,205,0.705244,0.516924,-0.673131,0.032012,-0.323076,...,-0.060627,-0.059381,-0.053645,-0.259468,0.252140,-0.138009,0.156007,-0.347590,0.188442,0.079837
70121,1358,9,ZZTop,Lovething,205,0.106758,0.353516,-0.619822,0.599401,-0.201721,...,0.289045,-0.010737,-0.168057,0.189666,0.265724,-0.035939,0.168925,-0.093011,0.274612,0.011483
70122,1358,7,ZZTop,Lovething,205,0.986733,0.684770,0.488515,0.581982,-0.586527,...,0.448924,0.245308,-0.528095,-0.282358,0.316982,-0.187706,-0.041834,0.117256,0.366200,0.208602


In [134]:
%%bigquery
CREATE MODEL binome_c05.kmeans_model
TRANSFORM (V0, V1, V2, V3, V4, V5, V6, V7, V8, V9, V10, V11, V12, V13, V14, V15)
OPTIONS(
  model_type='KMEANS',
  num_clusters=5,
  kmeans_init_method='KMEANS++',
  distance_type='euclidean',
  standardize_features=FALSE
) AS
WITH unfolded AS (
  SELECT
      vmd.* EXCEPT (vector),
      element,
      'V' || CAST(position AS STRING) AS position
  FROM binome_c05.vectors_with_metadata AS vmd
  CROSS JOIN UNNEST(vmd.vector) AS element WITH OFFSET position
)
SELECT *
FROM unfolded
PIVOT (
  MIN(element)
  FOR position IN (
    'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
  )
);

Query is running:   0%|          |

""


## Inférence du modèle

Appliquer ensuite la fonction d’inférence sur une requête pivot similaire appliquée à `flat_queries`, et observer le résultat.

Noter que le résultat ne peut pas servir à identifier facilement un morceau, car on obtient la distance aux centroïdes du modèle, qu'on ne sait pas vraiment interpréter...

In [135]:
%%bigquery
WITH unfolded AS (
  SELECT
      vmd.* EXCEPT (vector),
      element,
      'V' || CAST(position AS STRING) AS position
  FROM binome_c05.flat_queries AS vmd
  CROSS JOIN UNNEST(vmd.vector) AS element WITH OFFSET position
)
SELECT *
FROM unfolded
PIVOT (
  MIN(element)
  FOR position IN (
    'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
  )
);

Query is running:   0%|          |

Downloading:   0%|          |

,id,window_id,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,1,4,-1.481087,-0.160462,1.221947,0.374180,1.204532,-0.271382,0.331735,0.482447,-0.484102,-0.284827,-0.055476,-0.283090,-0.358441,0.121087,0.073820,0.121019
1,1,14,-0.958503,2.640996,-3.042559,-2.236541,0.486519,0.864159,-0.935728,-0.468577,1.408782,0.047250,-0.755446,-0.242191,-0.520071,0.359693,0.227907,0.269153
2,2,31,-0.580603,2.295242,-2.949005,-0.520440,-0.266236,0.594363,-0.469539,-0.027704,-0.124761,0.742902,0.208270,-0.038740,0.160211,-0.125833,0.341245,-0.226339
3,2,64,-0.439708,2.751366,-0.388683,-0.179845,-0.237591,-0.614449,0.857418,-0.299875,0.345018,-1.198934,-0.316748,0.296875,-0.289641,0.459883,0.388697,1.254033
4,2,121,-0.532996,-0.840476,0.517200,1.337992,1.540929,0.084688,0.373260,-0.671368,-0.042204,0.112618,-0.053218,-0.942988,-0.126161,0.118743,-0.080222,-0.102418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2,182,-0.713258,-3.200182,0.845827,2.373783,0.264961,1.187011,-1.290969,0.622488,-0.352651,0.124911,0.086592,0.490372,0.488991,0.271241,0.350576,0.189468
252,2,69,-0.562056,-2.664468,1.213531,-1.305584,-0.624682,-0.650292,-0.420388,0.559802,0.716897,1.203671,0.104020,0.075617,-0.815807,-0.649772,0.373307,-0.031283
253,2,146,-0.423297,2.532443,-0.182652,-0.670077,-0.375567,1.460377,-0.670416,-0.911831,1.078634,-0.919023,0.727275,-0.672183,-0.751550,0.335965,0.429393,0.033884
254,3,0,0.642043,0.451209,-0.051780,-0.662119,1.505607,-0.200581,0.567264,-0.121752,-0.364204,1.184966,-0.208099,-0.267309,-0.051448,0.063225,-0.062953,-0.103557


In [137]:
%%bigquery
SELECT * 
FROM ML.PREDICT(
  MODEL `binome_c05.kmeans_model`,
  (
    WITH unfolded AS (
      SELECT
          fq.id AS query_id,  
          fq.window_id AS window_id_query, 
          element,
          'V' || CAST(position AS STRING) AS position
      FROM binome_c05.flat_queries AS fq
      CROSS JOIN UNNEST(fq.vector) AS element WITH OFFSET position
    )
    SELECT *
    FROM unfolded
    PIVOT (
      MIN(element)
      FOR position IN (
        'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
      )
    )
  )
);

Query is running:   0%|          |

Downloading:   0%|          |

,CENTROID_ID,NEAREST_CENTROIDS_DISTANCE,query_id,window_id_query,V0,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.072367149557...",2,141,-0.926852,-3.331374,-0.645945,-0.010478,-0.329779,-0.831995,-1.331991,-0.658105,0.189859,0.075460,-0.540843,0.719058,0.643779,-0.708841,-0.333519,0.389430
1,2,"[{'CENTROID_ID': 2, 'DISTANCE': 2.425504976912...",2,116,-0.649272,-0.154460,0.013606,-0.018774,1.541192,-0.170556,0.695306,-0.683179,0.483031,-0.212100,1.225893,-0.321000,-0.033554,-0.328441,-0.275308,0.549008
2,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.095559165038...",2,150,-0.636946,2.043043,-2.426071,-0.624738,0.311390,-0.921696,-1.280427,-0.261394,-1.340678,0.514016,-0.302289,-0.905542,0.546515,0.386349,0.247097,0.156320
3,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.049125810091...",2,147,-0.328674,3.367298,0.753207,0.760437,0.780132,-0.446419,0.653001,-0.222031,0.952972,-0.363487,-0.089566,0.400881,0.021221,0.938632,0.033665,0.289885
4,2,"[{'CENTROID_ID': 2, 'DISTANCE': 3.374220736626...",2,139,-0.602187,-2.950732,0.213952,-0.853494,-0.285992,-0.301427,-0.114543,0.825084,-0.030278,0.379530,0.337610,0.483817,-0.229247,-0.530568,0.158166,-0.014632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,2,"[{'CENTROID_ID': 2, 'DISTANCE': 4.342453022240...",2,18,-0.756687,-3.246436,0.044029,0.331182,-2.713981,-0.287636,0.537776,-0.227857,0.252245,0.221432,-0.176453,-0.164392,0.273547,0.040151,-0.281958,0.100792
252,2,"[{'CENTROID_ID': 2, 'DISTANCE': 3.988823311950...",2,31,-0.580603,2.295242,-2.949005,-0.520440,-0.266236,0.594363,-0.469539,-0.027704,-0.124761,0.742902,0.208270,-0.038740,0.160211,-0.125833,0.341245,-0.226339
253,2,"[{'CENTROID_ID': 2, 'DISTANCE': 2.815783103658...",2,45,-0.755233,-1.280688,-0.357910,1.913668,0.680093,0.336219,-0.380230,-0.113066,-0.656329,-0.809868,-0.303010,-0.495088,0.383911,0.164991,0.271422,0.144584
254,2,"[{'CENTROID_ID': 2, 'DISTANCE': 2.989565975404...",2,161,-0.577635,2.003849,-1.613415,0.030223,0.545317,-0.827999,0.070237,0.584958,-0.210530,0.347383,0.303657,0.510695,0.272015,0.026312,-0.272701,-0.354281


## Exploitation du clustering sur la base de référence

### Récupération
Appliquer l'inférence non plus sur `flat_queries`, mais sur `vectors_with_metadata` elle-même (la table qui a servi à l'entraînement). Récupérer le résultat dans un dataframe.

In [138]:
%%bigquery clustering_vmd
SELECT * 
FROM ML.PREDICT(
  MODEL `binome_c05.kmeans_model`,
  (
    WITH unfolded AS (
      SELECT
          vmd.*,  
          element,
          'V' || CAST(position AS STRING) AS position
      FROM binome_c05.vectors_with_metadata AS vmd
      CROSS JOIN UNNEST(vmd.vector) AS element WITH OFFSET position
    )
    SELECT *
    FROM unfolded
    PIVOT (
      MIN(element)
      FOR position IN (
        'V0', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14', 'V15'
      )
    )
  )
);

Query is running:   0%|          |

Downloading:   0%|          |

Observer la structure du dataframe. Deux informations vont nous intéresser :
- `song_id`, `window_id`, `artist`, etc. : informations de la fenêtre provenant d'un morceau classé
- `CENTROID_ID` : n° du cluster auquel appartient la fenêtre

Dans la suite, on ne fait plus appel à BigQuery mais on manipule le dataframe mémorisé à l'étape précédente, avec Pandas.

Construire un nouveau dataframe avec une ligne par morceau et les colonnes suivantes :
- `song_id` : ID du morceau
- `artist` : nom de l'artiste
- `centroid_id` : n° du cluster le plus fréquent pour ce morceau (associé au plus grand nombre de fenêtres)

Indications : faire un regroupement du dataframe par morceau et artiste, et appliquer au regroupement une fonction qui détermine le "mode" de la série `CENTROID_ID` (doc Pandas ici : https://pandas.pydata.org/docs/reference/api/pandas.Series.mode.html).

In [145]:
import pandas as pd
df = (
    clustering_vmd
    .groupby(["song_id", "artist"])["CENTROID_ID"]
    .agg(lambda x: x.mode().iloc[0])
    .reset_index()
    .rename(columns={"CENTROID_ID": "centroid_id"})
)
print(df_final.head())

   song_id        artist  centroid_id
0        1       Raphael            2
1        2     Indochine            2
2        3  W. A. Mozart            2
3        4     Indochine            2
4        5  W. A. Mozart            2


Utiliser ensuite ce dataframe pour répondre aux questions suivantes (**merci de donner le code Python en plus des réponses brutes**).

**Question** : Quelles sont les nombres de morceaux par cluster ? Comment interpréter le fait que certains clusters ne soient pas du tout représentés ?

**Question** : Quels sont les artistes du cluster n°7, avec combien de morceaux chacun ?

**Question** : De la même manière qu'on a déterminé le cluster le plus représenté pour chaque morceau, classer les artistes par cluster le plus représenté pour leurs morceaux (en utilisant le dernier dataframe construit donc).

**Question** : Le clustering des artistes n'est pas très convaincant (si vous les connaissez aussi, vous constaterez que les clusters mélangent un peu tout et n'importe quoi). En reconsidérant tout le cas d'usage, proposer des raisons possibles. **Vous pouvez répondre à cette question même si vous n'avez pas fait les précédentes !**

1 - Le cluster avec l'identifiant 2 contient 1240 morceaux, tandis que le cluster 4 contient 127 morceaux.